## Eigensolving

One of the main uses of dynamite is solving for eigenvalues and eigenstates---whether the ground state, or the middle of the spectrum. Here we will explore how to do that.

### Ground state of the transverse field Ising model

We revisit our favorite Hamiltonian, the TFIM:

$$H = J \sum_i \sigma^z_i \sigma^z_{i+1} + h \sum_i \sigma^x_i$$

In [ ]:
# always convenient to globally set the length of the spin chain
from dynamite import config
config.L = 12

In [ ]:
from dynamite.operators import sigmax, sigmay, sigmaz
from dynamite.operators import index_sum

def tfim(J, h):
    ising = J*index_sum(sigmaz(0)*sigmaz(1), boundary='closed')
    field = h*index_sum(sigmax(0))
    return ising + field

Let's find the ground state of this Hamiltonian! To do so, you can use the `Operator.eigsolve()` method:

In [ ]:
H = tfim(J=1, h=0.1)
eigvals = H.eigsolve()

print(eigvals)

By default, dynamite solves for the ground state energy. Sometimes it returns a few of the low-lying excited states too, if it happened to converge them during the solve.

You can request the energy of a few states instead of just the ground state:

In [ ]:
eigvals = H.eigsolve(nev=3)
print(eigvals)

dynamite can also return the eigenvectors, as a State object. To do this, just pass the `getvecs=True` flag:

In [ ]:
eigvals, eigvecs = H.eigsolve(getvecs=True)
print(eigvecs)

dynamite always returns a list of eigenvalues and eigenvectors; if you want the ground state, it will be the first in the list:

In [ ]:
gs = eigvecs[0]
gs_energy = eigvals[0]

In [ ]:
# Exercise: confirm that the expectation value of the Hamiltonian is equal to the eigenvalue


Now that we have the ground state, let's compute some measurements on it! First, let's measure $\langle \psi | \sigma_i^z |\psi \rangle$ and plot it:

In [ ]:
def measure_sigmaz(i, state):
    return state.dot(sigmaz(i)*state)

# measurements
z_vals = []
for i in range(config.L):
    z_vals.append(measure_sigmaz(i, gs))

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

def plot_measmts(vals):
    # color of the dots is defined by the measurement values
    s = plt.scatter(range(config.L), [1 for i in range(config.L)], c=np.real(z_vals), s=50)

    plt.title('$<S^z>$')
    plt.xticks([])
    plt.yticks([])

    plt.colorbar(s, ax=plt.gca())
    
plot_measmts(z_vals)

We see that the ground state is antiferromagnetic---this makes sense, since we set $J$ to be positive.

Now try implementing the two-site correlator $\langle \psi | \sigma_i^z \sigma_j^z | \psi \rangle - \langle \psi | \sigma_i^z | \psi \rangle \langle \psi | \sigma_j^z | \psi \rangle $! If you really want to have fun, make a 2D scatter plot of the correlator between each pair of sites.

In [ ]:
# Exercise: implement the two-site correlator, and plot it!


OK, next exercise: let's look at the phase transition! Remember, if we add a small symmetry breaking term (like a weak $Z$ field), we expect the total magnetization $\sum_i \sigma_i^z$ to be large on one side of the phase transition, and go to zero on the other side. Your goal here is to solve for the ground state for many different values of $h$, calculating the total magnetization for each one. Then plot that total magnetization as a function of $h$!

In [ ]:
# here is a function to measure the total magnetization of a state, to get you started!

def measure_mtot(state):
    mtot = index_sum(sigmaz(0))
    exp_value = state.dot(mtot*state)
    return exp_value.real  # only need the real part, it's Hermitian

In [ ]:
# Exercise: sweep h through the phase transition, and measure the total magnetization of the ground state
# at each step. You can use the tfim(J, h) function from earlier to build the Hamiltonian, but 
# don't forget to add a weak (maybe order 1e-5) symmetry-breaking Z-field! 
# Also, don't forget that J must be negative to see the ferromagnet.

# Then you can use matplotlib to plot the magnetization through the transition!


### Calculating interior eigvalues

dynamite can calculate more than just the ground state---it can also solve for portions of the interior eigenvalues. Do note that doing so is significantly slower and uses more memory than solving for ground states---but it's still way more efficient than solving for the full spectrum, if all you need is a few states in the middle of the spectrum.

**Important**: do *not* use dynamite's eigensolver to try to solve for the whole spectrum---that's not what it's designed for, and it will not work well. If you need all the eigenvalues/eigenstates, it is faster to just convert the operator to a dense NumPy matrix by calling `.to_numpy(sparse=False)`, and then use NumPy or SciPy's eigensolvers on the result. 

To calculate interior eigenvalues, we pass the `target` parameter to `.eigsolve`. This will solve for the eigenvalue (or set of `nev` eigenvalues) nearest in value to the target. For example, to solve for the 10 eigenpairs with eigenvalues closest to -0.5:

In [ ]:
H = tfim(-1, 0.5) + 1e-6*index_sum(sigmaz())

eigvals, eigvecs = H.eigsolve(target=-0.5, nev=10, getvecs=True)

print(eigvals)

And as expected the total magnetization of a highly excited state is less than that of the ground state:

In [ ]:
eigvals, eigvecs = H.eigsolve(getvecs=True)  # solve for the ground state
ground_state = eigvecs[0]
print('ground state magnetization:         ', measure_mtot(ground_state))

eigvals, eigvecs = H.eigsolve(target=-0.1, getvecs=True)  # solve for excited state near energy -0.1
excited_state = eigvecs[0]
print('highly excited state magnetization: ', measure_mtot(excited_state))

## Up next

[Continue to notebook 4](4-TimeEvolution.ipynb) to learn how to compute time evolution under a Hamiltonian.